# Extract Stock Ticker Mentions from Wallstreet Bets Data

In [1]:
#import dependencies
import csv
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt

In [2]:
# import Wallstreet Bets csv file from Kaggle
# url= https://www.kaggle.com/gpreda/reddit-wallstreetsbets-posts
csvfile = "./Resources/reddit_wsb.csv"
wsb_csv = pd.read_csv(csvfile)
wsb_csv = pd.read_csv(csvfile, parse_dates=['timestamp'], infer_datetime_format=True)
wsb_csv.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


In [3]:
wsb_csv.dtypes

title                object
score                 int64
id                   object
url                  object
comms_num             int64
created             float64
body                 object
timestamp    datetime64[ns]
dtype: object

In [4]:
#replace body n/a with empty string
wsb_csv['body'] = wsb_csv['body'].fillna("")

#merge title and body together for comprehensive field search
wsb_csv['title_body'] = wsb_csv['title'] + wsb_csv['body']

#drop unneeded columns
wsb_csv.drop(columns=['score', 'id','url','comms_num','created', 'title', 'body'], inplace=True)

#convert search field to uppercase to allow desensitize case for searching
wsb_csv['title_body'] = wsb_csv['title_body'].str.upper()

#drop time from date timestamp
wsb_csv['timestamp'] = pd.to_datetime(wsb_csv['timestamp']).dt.date

#add weekday to df, need to change timestamp to mirror with stock ticker data
wsb_csv['weekday'] = pd.to_datetime(wsb_csv['timestamp']).dt.day_name()

wsb_csv.head()

,timestamp,title_body,weekday
0,2021-01-28,"IT'S NOT ABOUT THE MONEY, IT'S ABOUT SENDING A...",Thursday
1,2021-01-28,MATH PROFESSOR SCOTT STEINER SAYS THE NUMBERS ...,Thursday
2,2021-01-28,EXIT THE SYSTEMTHE CEO OF NASDAQ PUSHED TO HAL...,Thursday
3,2021-01-28,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,Thursday
4,2021-01-28,"NOT TO DISTRACT FROM GME, JUST THOUGHT OUR AMC...",Thursday


In [5]:
#create dictionary with term list for each ticker
ticker_list = {'AMC': ['AMC'], 'GME': ['GME'], 'CLOV': ['CLOV'], 'BB': ['BB'], 'CLF': ['CLF'], 'CLNE': ['CLNE'],
               'WKHS': ['WKHS'], 'SENS': ['SENS'], 'PLTR': ['PLTR'], 'RKT': ['RKT'], 'WWE': ['WWE'], 'FORD': ['FORD']}

#loop through dataframe to identify number of mentions for each ticker
for ticker, stock_names in ticker_list.items():
    for name in stock_names:
        wsb_csv[ticker] = wsb_csv['title_body'].str.contains(name).astype(int)

wsb_csv.head()

,timestamp,title_body,weekday,AMC,GME,CLOV,BB,CLF,CLNE,WKHS,SENS,PLTR,RKT,WWE,FORD
0,2021-01-28,"IT'S NOT ABOUT THE MONEY, IT'S ABOUT SENDING A...",Thursday,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-01-28,MATH PROFESSOR SCOTT STEINER SAYS THE NUMBERS ...,Thursday,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-01-28,EXIT THE SYSTEMTHE CEO OF NASDAQ PUSHED TO HAL...,Thursday,0,1,0,0,0,0,0,0,0,0,0,0
3,2021-01-28,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,Thursday,0,1,0,0,0,0,0,0,0,0,0,0
4,2021-01-28,"NOT TO DISTRACT FROM GME, JUST THOUGHT OUR AMC...",Thursday,1,1,0,0,0,0,0,0,0,0,0,0


In [6]:
#group number of mentions by the date mentioned, move date from index to column
mentions_by_ticker_df = wsb_csv.groupby(["timestamp"]).sum()
mentions_by_ticker_df.reset_index(inplace=True)
mentions_by_ticker_df['timestamp'] = mentions_by_ticker_df['timestamp'].astype('datetime64[ns]')
mentions_by_ticker_df

,timestamp,AMC,GME,CLOV,BB,CLF,CLNE,WKHS,SENS,PLTR,RKT,WWE,FORD
0,2020-09-29,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-01-28,214,419,1,123,0,0,0,11,7,0,1,7
2,2021-01-29,2276,4677,0,1488,0,0,0,87,41,4,3,97
3,2021-01-30,151,528,0,117,0,0,1,23,8,2,1,30
4,2021-01-31,98,410,0,84,1,0,0,25,7,0,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2021-05-28,38,13,0,19,0,0,0,7,4,4,0,9
119,2021-05-29,14,2,2,6,0,1,0,2,0,1,0,1
120,2021-05-30,10,8,0,7,0,0,0,6,1,0,0,2
121,2021-05-31,3,5,0,8,0,0,0,2,0,0,0,5


In [7]:
#chose list of popular stocks from url = https://memestocks.org

#unpivot columns using Pandas melt to consolidate all mentions into one column
columns = ['AMC', 'GME', 'CLOV', 'BB', 'CLF', 'CLNE', 'WKHS', 'SENS', 'PLTR', 'RKT', 'WWE', 'FORD']
consolidated_mentions = pd.melt(mentions_by_ticker_df, id_vars=['timestamp'], value_vars = columns,
                               var_name='ticker', value_name='mention_count')
consolidated_mentions

,timestamp,ticker,mention_count
0,2020-09-29,AMC,0
1,2021-01-28,AMC,214
2,2021-01-29,AMC,2276
3,2021-01-30,AMC,151
4,2021-01-31,AMC,98
...,...,...,...
1471,2021-05-28,FORD,9
1472,2021-05-29,FORD,1
1473,2021-05-30,FORD,2
1474,2021-05-31,FORD,5


In [8]:
# delete all rows mention count equal to zero
indexNames = consolidated_mentions[consolidated_mentions['mention_count'] ==0].index
consolidated_mentions.drop(indexNames, inplace=True)

#reset the index after dropping zeroes
consolidated_mentions.reset_index(drop=True)

,timestamp,ticker,mention_count
0,2021-01-28,AMC,214
1,2021-01-29,AMC,2276
2,2021-01-30,AMC,151
3,2021-01-31,AMC,98
4,2021-02-01,AMC,89
...,...,...,...
912,2021-05-26,FORD,2
913,2021-05-28,FORD,9
914,2021-05-29,FORD,1
915,2021-05-30,FORD,2


In [9]:
consolidated_mentions.head()

,timestamp,ticker,mention_count
1,2021-01-28,AMC,214
2,2021-01-29,AMC,2276
3,2021-01-30,AMC,151
4,2021-01-31,AMC,98
5,2021-02-01,AMC,89


In [10]:
#add weekday to df, need to change timestamp to mirror with stock ticker data
consolidated_mentions['weekday'] = pd.to_datetime(consolidated_mentions['timestamp']).dt.day_name()

#create a new column to identify weekends and how many days to add
consolidated_mentions["days_to_add"] = 0

#add 2 days on Saturday to match to Monday's stock data
consolidated_mentions.loc[consolidated_mentions.weekday=="Saturday", 'days_to_add'] = 2

#add 1 day to Sunday to match to Monday's stock data
consolidated_mentions.loc[consolidated_mentions.weekday=="Sunday", 'days_to_add'] = 1

#add days to holidays stock data
consolidated_mentions.loc[consolidated_mentions.timestamp=='2021-02-15', 'days_to_add'] = 1
consolidated_mentions.loc[consolidated_mentions.timestamp=='2021-04-02', 'days_to_add'] = 3
consolidated_mentions.loc[consolidated_mentions.timestamp=='2021-05-31', 'days_to_add'] = 1

In [11]:
#adjust mention date based on days_to_add, this is to line up with the Yahoo Finance data later
temp = consolidated_mentions['days_to_add'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
consolidated_mentions['AdjDate'] = consolidated_mentions['timestamp'] + temp

#review changes to ensure accuracy
consolidated_mentions.groupby('weekday').max('days_to_add')

,mention_count,days_to_add
weekday,,
Friday,4677,3
Monday,342,1
Saturday,939,2
Sunday,410,1
Thursday,544,0
Tuesday,450,0
Wednesday,738,0


In [12]:
#export mention count from Wallstreet Bets to csv
consolidated_mentions.to_csv("Stock_Mention_Count.csv")

# Extract stock ticker close prices from Yahoo Finance

In [23]:
#download stock ticker data from Jan 1-Jun 1 (same time period as Wallstreet Bets)
data = yf.download('GME AMC CLOV BB CLF CLNE WKHS SENS PLTR RKT WWE FORD', start="2021-01-01", end="2021-06-01")
data.head()

[*********************100%***********************]  12 of 12 completed


Adj Close                                                  \
                 AMC    BB        CLF   CLNE   CLOV  FORD        GME   
Date                                                                   
2021-01-04      2.01  6.58  14.420000   7.83  15.61  1.88  17.250000   
2021-01-05      1.98  6.77  15.700000   8.68  15.09  1.89  17.370001   
2021-01-06      2.01  6.71  17.420000   9.00  14.53  1.85  18.360001   
2021-01-07      2.05  7.06  17.670000  11.10  16.02  1.91  18.080000   
2021-01-08      2.14  7.56  18.040001  10.25  15.90  1.92  17.690001   

                                         ...    Volume                      \
                 PLTR        RKT   SENS  ...       CLF      CLNE      CLOV   
Date                                     ...                                 
2021-01-04  23.370001  19.115538  0.930  ...  13943600   6023300  21001700   
2021-01-05  24.600000  19.010401  0.950  ...  17867500  13054700  13198800   
2021-01-06  23.540001  18.494284  0.898  ...  27988700  13370400  26843600   
2021-01-07  25.000000  18.847919  0.880  ...  18574800  21143600  20563800   
2021-01-08  25.200001  19.067749  0.867  ...  15849300  17507800  15380000   

                                                                              
              FORD       GME      PLTR       RKT      SENS      WKHS     WWE  
Date                                                                          
2021-01-04  117800  10022500  44970400   6486900   9844900  19044700  744800  
2021-01-05  123100   4961500  29050400  14297700  31283500  13573300  549800  
2021-01-06  322000   6056200  32732900  16289300   9545400  18997800  740700  
2021-01-07   95200   6129300  32240000   8838300   8125700  24390400  573900  
2021-01-08   84200   6482000  41313800   6596800   4864000  21290600  574200  

[5 rows x 72 columns]

In [24]:
#drop columns not needed
data.drop(columns=["Close", "High", "Low", "Open", "Volume"], inplace=True)
data.head()

Adj Close                                                  \
                 AMC    BB        CLF   CLNE   CLOV  FORD        GME   
Date                                                                   
2021-01-04      2.01  6.58  14.420000   7.83  15.61  1.88  17.250000   
2021-01-05      1.98  6.77  15.700000   8.68  15.09  1.89  17.370001   
2021-01-06      2.01  6.71  17.420000   9.00  14.53  1.85  18.360001   
2021-01-07      2.05  7.06  17.670000  11.10  16.02  1.91  18.080000   
2021-01-08      2.14  7.56  18.040001  10.25  15.90  1.92  17.690001   

                                                               
                 PLTR        RKT   SENS       WKHS        WWE  
Date                                                           
2021-01-04  23.370001  19.115538  0.930  21.420000  47.122772  
2021-01-05  24.600000  19.010401  0.950  22.430000  47.921124  
2021-01-06  23.540001  18.494284  0.898  23.650000  48.400135  
2021-01-07  25.000000  18.847919  0.880  27.600000  48.330280  
2021-01-08  25.200001  19.067749  0.867  25.565001  48.140675

In [25]:
data.columns = data.columns.droplevel(0)
data.reset_index(inplace=True)
data

,Date,AMC,BB,CLF,CLNE,CLOV,FORD,GME,PLTR,RKT,SENS,WKHS,WWE
0,2021-01-04,2.010000,6.58,14.420000,7.83,15.61,1.88,17.250000,23.370001,19.115538,0.930,21.420000,47.122772
1,2021-01-05,1.980000,6.77,15.700000,8.68,15.09,1.89,17.370001,24.600000,19.010401,0.950,22.430000,47.921124
2,2021-01-06,2.010000,6.71,17.420000,9.00,14.53,1.85,18.360001,23.540001,18.494284,0.898,23.650000,48.400135
3,2021-01-07,2.050000,7.06,17.670000,11.10,16.02,1.91,18.080000,25.000000,18.847919,0.880,27.600000,48.330280
4,2021-01-08,2.140000,7.56,18.040001,10.25,15.90,1.92,17.690001,25.200001,19.067749,0.867,25.565001,48.140675
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2021-05-24,13.680000,8.62,18.559999,7.96,6.92,2.53,180.009995,21.219999,16.900000,1.990,8.310000,57.919998
98,2021-05-25,16.410000,8.59,18.200001,7.63,7.02,2.40,209.429993,21.480000,16.809999,1.860,8.140000,57.279999
99,2021-05-26,19.559999,9.44,18.920000,8.04,7.33,2.44,242.559998,22.120001,17.459999,1.970,9.130000,56.880001
100,2021-05-27,26.520000,9.97,20.200001,8.09,7.83,2.83,254.130005,22.700001,18.350000,2.070,9.620000,56.669998


In [26]:
#use natural log from numpy to get stock price changes
data['AMC_pct_chg']=np.log(data['AMC']/data['AMC'].shift(1))
data['BB_pct_chg']=np.log(data['BB']/data['BB'].shift(1))
data['CLF_pct_chg']=np.log(data['CLF']/data['CLF'].shift(1))
data['CLNE_pct_chg']=np.log(data['CLNE']/data['CLNE'].shift(1))
data['CLOV_pct_chg']=np.log(data['CLOV']/data['CLOV'].shift(1))
data['FORD_pct_chg']=np.log(data['FORD']/data['FORD'].shift(1))
data['GME_pct_chg']=np.log(data['GME']/data['GME'].shift(1))
data['PLTR_pct_chg']=np.log(data['PLTR']/data['PLTR'].shift(1))
data['RKT_pct_chg']=np.log(data['RKT']/data['RKT'].shift(1))
data['SENS_pct_chg']=np.log(data['SENS']/data['SENS'].shift(1))
data['WKHS_pct_chg']=np.log(data['WKHS']/data['WKHS'].shift(1))
data['WWE_pct_chg']=np.log(data['WWE']/data['WWE'].shift(1))

In [27]:
#drop unneeded columns with original adjusted close price
yf_columns = ['AMC', 'BB', 'CLF', 'CLNE', 'CLOV', 'FORD', 'GME', 'PLTR', 'RKT', 'SENS', 'WKHS', 'WWE']
data.drop(columns=yf_columns, inplace=True)

In [28]:
#unpivot columns using Pandas melt to consolidate all mentions into one column
columns = ['AMC_pct_chg', 'BB_pct_chg', 'CLF_pct_chg', 'CLNE_pct_chg', 'CLOV_pct_chg', 'FORD_pct_chg', 'GME_pct_chg',
           'PLTR_pct_chg', 'RKT_pct_chg', 'SENS_pct_chg', 'WKHS_pct_chg', 'WWE_pct_chg']
consolidated_yf_data = pd.melt(data, id_vars=['Date'], value_vars = columns,
                               var_name='ticker', value_name='percent_change')

#first day of each ticker data is n/a, replace with zeroes
consolidated_yf_data['percent_change'] = consolidated_yf_data['percent_change'].fillna(0)

#trim ticker names
consolidated_yf_data['ticker']=consolidated_yf_data['ticker'].str.replace('_pct_chg', '')

consolidated_yf_data

,Date,ticker,percent_change
0,2021-01-04,AMC,0.000000
1,2021-01-05,AMC,-0.015038
2,2021-01-06,AMC,0.015038
3,2021-01-07,AMC,0.019705
4,2021-01-08,AMC,0.042966
...,...,...,...
1219,2021-05-24,WWE,0.006409
1220,2021-05-25,WWE,-0.011111
1221,2021-05-26,WWE,-0.007008
1222,2021-05-27,WWE,-0.003699


In [29]:
#export Yahoo Finance stock price changes to csv
consolidated_yf_data.to_csv("Stock_Price_Changes.csv")